<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ClubRaiders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

1. Welcome to Club Raiders.
2. 'The Club' or 'The Cabal' is a mysterious group in the game Elite: Dangerous.
3. The Club is apparently related to two other long-standing mysteries in Elite: Dangerous - Raxxla and The Dark Wheel.
4. Almost everything we know about The Club is conjecture.
5. A group in Elite Dangerous known as The Children of Raxxla has provided reliable information about other in-game mysteries in the past.
6. The Children of Raxxla have released documents about The Club.  This information may or may not be reliable.
7. Two of these documents are known as The Holdstock Report and The Club Unmasked.
8. These two documents suggest that some in-game minor factions support The Club, and others oppose them.

This document considers the factions identified on the chart below, in particular where these factions are operating throughout the Milky Way.

![Big Crazy Club Diagram](https://github.com/HausReport/ClubRaiders/blob/master/info/My%20Club%20Diagram.png?raw=true)

Helping individual players and groups find Club factions operating in their vicinity is part one of the Bolan Plan: **Identify, Engage, Eradicate**.


# Number Crunching

In [1]:
#@title
import sys
sys.executable
sys.path

%load_ext google.colab.data_table
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"



  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-ds7tfeec/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-ds7tfeec/craid
  Created wheel for craid: filename=craid-0.37-cp36-none-any.whl size=11444 sha256=9e72a5ba0a1d2b81a101896c6abf097c6d8a429ad38d83545b284fff880d9b17
  Stored in directory: /tmp/pip-ephem-wheel-cache-wuvv7i8j/wheels/59/74/32/b75ca6dbfddc83071fd4a4a492a26c1066d06840be16c7d535
Successfully built craid


In [0]:
#@title 
import craid.Club
import craid.LoadDataFromEDDB
from craid.eddb.FactionInstance import FactionInstance
import craid.Crunch
craid.LoadDataFromEDDB.load_data()
club_systems_arr: FactionInstance = craid.Crunch.getSystemsArray()
df = craid.Crunch.getDataFrame(club_systems_arr)
#
# Dataframe of all club factions except Emp Grace
#
filter = df[ ~df[ 'factionName' ].str.contains("Emperor's Grace") ]


# Intercepted Message

In [3]:
#@title 
from dateutil.relativedelta import relativedelta
import datetime
from datetime import timezone 

frame = df

dt = datetime.datetime.now() 
utc_time = dt.replace(tzinfo = timezone.utc) 

new_date = utc_time + relativedelta(years=1)
print("TO: CMDR [REDACTED]/OPS")
print("FM: CMDR E[REDACTED]/MAIA DATA")
print("DT: " + new_date.strftime("%d-%b-%Y %H:%M"))
print("RE: HAXDAT/CLUB STUFF TO SHOOT")
print("===================================")

print()
print("Hey.  Spent the last two nights up to my ears in data from the last breach.  Here are the ")
print("highlights.  I think they're pretty compelling, but I'm juiced to the gills to stay awake.")

print()

print( "Club factions are currently active in " + str(frame['systemName'].count()) +" systems, home to " + '{:,}'.format(int(frame['population'].sum(axis=0))) + " souls." )
idx = frame['x'].idxmin()

print('They span the galaxy from {0} in {1}, {2} light years west of Sol'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))
idx = frame['x'].idxmax()

print('to {0} in {1}, {2} light years to the east.'.format(frame.iloc[idx,1], frame.iloc[idx,0], abs(int(frame.iloc[idx,2]))))

print()

print("It's encouraging to note that club factions only control {0} of these systems, or about {1:.0%}.".format(frame['control'].sum(), frame['control'].sum()/frame['systemName'].count()))
print("Their influence in all systems they operate in ranges from {0:.0%} to {1:.0%}, averaging {2:.0%}, but".format(df['influence'].min()/100.0, df['influence'].max()/100.0, df['influence'].mean()/100.0))

uncontrolled = frame[ ~frame[ 'control' ]]

print("in systems they don't control, these numbers fall to {0:.0%}-{1:.0%} influence with an average of only {2:.0%}.".format(uncontrolled['influence'].min()/100.0, uncontrolled['influence'].max()/100.0, uncontrolled['influence'].mean()/100.0))

gaxx =uncontrolled['influence'].describe()

print("In these non-control systems, half of their factions have less than {0:.0%} influence and a full\nquarter are operating at less than {1:.0%}.".format(gaxx[5]/100,gaxx[4]/100))

print()
print("As you know, we forced a number of club retreats lately.  Here's today's hot list:")
print()

gg = filter.copy()
gg = gg[gg['influence']<6]
gg = gg[gg['influence']>0]
gg = gg[~gg['isHomeSystem']]
gg=  gg.sort_values('influence')
print(gg.to_string(columns=['factionName','systemName','influence','vulnerable']))


print()
print("In other words: there's some low-hanging fruit to pick out there.")


print()
print("You said you just wanted something to shoot at.  Go shoot 'em.  I should have the new system")
print("online in a few days.  Let me know if you want another update after we xxxxxxx the new terminal")
print("tonight.  In the meantime, I'm gonna cause some trouble out here in the Pleiades.")

print()
print("Fly dangerous,")
print("-E")

TO: CMDR [REDACTED]/OPS
FM: CMDR E[REDACTED]/MAIA DATA
DT: 22-May-2021 19:49
RE: HAXDAT/CLUB STUFF TO SHOOT

Hey.  Spent the last two nights up to my ears in data from the last breach.  Here are the 
highlights.  I think they're pretty compelling, but I'm juiced to the gills to stay awake.

Club factions are currently active in 713 systems, home to 335,698,955,020 souls.
They span the galaxy from Sirius Corporation in Sothis, 352 light years west of Sol
to Gallant Investment Brokers in Exphiay, 283 light years to the east.

It's encouraging to note that club factions only control 145 of these systems, or about 20%.
Their influence in all systems they operate in ranges from 0% to 100%, averaging 20%, but
in systems they don't control, these numbers fall to 0%-41% influence with an average of only 12%.
In these non-control systems, half of their factions have less than 11% influence and a full
quarter are operating at less than 8%.

As you know, we forced a number of club retreats lately

# Systems with Club Influence by Allegiance

The graph below shows systems where Club factions are operating.  The ball of blue systems is largely Emperor's Grace factions that may or may not have much to do with Club operations.

In [4]:
import plotly.express as px
fig = px.scatter_3d(df, x='x', y='z', z='y',color='influence', hover_data={  'systemName', 'factionName'})
fig.show()

If we remove the Emperor's Grace factions for a moment, it will give a clearer picture of the other factions in play.

In [5]:
figx = px.scatter_3d(filter, x='x', y='z', z='y',color='allegiance', hover_data={  'systemName', 'factionName'})
figx.show()

The table below presents the same data in a tabular format.  


In [6]:
df

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable
0,27 G. Caeli,Vodyanes Emperor's Grace,75.84375,-62.09375,-33.34375,Empire,False,3993953406,18.6000,2020-05-22 02:20:42,False,
1,64 Ceti,Worster Insurance,-37.06250,-100.06250,-79.12500,Independent,True,5096451,2.8000,2020-05-21 20:53:49,False,Low Influence
2,Abroin,Abroin Universal PLC,-94.53125,110.09375,-40.50000,Independent,False,2813462,42.0000,2020-05-20 02:22:14,True,
3,Achuar,HIP 5183 Emperor's Grace,26.09375,-231.43750,5.93750,Empire,False,2138,12.3016,2020-05-20 16:55:39,False,
4,Adachit,Rishair Emperor's Grace,107.28125,-163.84375,44.31250,Empire,False,8142435133,8.3083,2020-05-21 02:30:33,False,
...,...,...,...,...,...,...,...,...,...,...,...,...
708,Col 285 Sector BQ-N c7-13,Sirius Luxury Transports,158.31250,69.50000,-42.03125,Independent,False,24119,13.9000,2020-05-15 14:47:47,False,Anarchy
709,Col 285 Sector AQ-P d5-52,HIP 112001 Emperor's Grace,-12.18750,-207.25000,123.37500,Empire,False,28593,8.8088,2020-05-18 18:18:50,False,
710,Mel 22 Sector GM-V c2-8,Turner Research Group,-192.12500,-194.96875,-471.65625,Alliance,False,18550,100.0000,2020-05-21 22:00:31,True,
711,Delphi,Janus Incorporated,-63.59375,-147.40625,-319.09375,Independent,False,300000,8.4000,2020-05-22 01:43:07,False,


# More crunching

In [7]:
#@title 
import plotly.graph_objects as go

res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['y'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-20,
            x1= 160,
            y1= 100,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -175,
            x1= -50,
            y1= -75,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -150,
            x1= 300,
            y1= -100,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)




In [8]:
import plotly.graph_objects as go

res = [i + " in " + j for i, j in zip(filter['factionName'], filter['systemName'])] 
#%run TwoDimensionalRegionsPlot.py

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=filter['x'],
    y=filter['z'],
    mode="markers",  hovertext= res ))

fig.add_trace(go.Scatter(
    x=[-270, -270,  -95,  140,   -20,   -75,   70,   275,   30],
    y=[-200, -50,   150,  120,     0,  -175,   80,   -80,   120],
    mode="text",
    name="Regions",
    text=["Turner Reach", "Sirius Reach", "Corporate Way","Sirius East","Sirius Core","Pleiades", "The Old Worlds", "Gallant Beach", "Bentonia"],
    textposition="bottom center"
))

# Add shapes
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-350,
            y0=-175,
            x1= -175,
            y1= -250,
            opacity=0.2,
            fillcolor="blue",
            line_color="blue",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-375,
            y0=-100,
            x1= -125,
            y1= 50,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-110,
            y0=75,
            x1= -75,
            y1= 130,
            opacity=0.2,
            fillcolor="green",
            line_color="green",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=120,
            y0=-80,
            x1= 160,
            y1= 130,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-40,
            y0= -30,
            x1= 50,
            y1= 30,
            opacity=0.2,
            fillcolor="yellow",
            line_color="yellow",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=-100,
            y0= -400,
            x1= -50,
            y1= -200,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
       dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  60,
            y0= 10,
            x1= 80,
            y1= 60,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  250,
            y0= -40,
            x1= 300,
            y1= -0,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
        dict(
            type="rect",
            xref="x",
            yref="y",
            x0=  25,
            y0= 135,
            x1= 40,
            y1= 175,
            opacity=0.2,
            fillcolor="orange",
            line_color="orange",
        ),
    ],
)
fig.update_layout(showlegend=False)

# Galactic Areas of Club Influence

Again, setting aside the Emperor's Grace factions for clarity, there are around 10 general areas where Club factions tend to operate.

In [9]:
fig.show()

# All Club Factions as Comma-Delimited Text

If you like to work with data, you might find the comma-delimited text below useful:


In [10]:
#@title 
for xcs in club_systems_arr:
    cs: FactionInstance = xcs
    #if( "Emperor's Grace" in cs.get_name() ): continue
    #vulnerable = cs.isVulnerable()
    #if (vulnerable == False): continue
    cs.printCSV()

Vodyanes Emperor's Grace,27 G. Caeli,75.84,-62.09,-33.34,Empire,18.60,,22-May-2020 02:20
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.80,Low Influence,21-May-2020 20:53
Abroin Universal PLC,Abroin,-94.53,110.09,-40.50,Independent,42.00,,20-May-2020 02:22
HIP 5183 Emperor's Grace,Achuar,26.09,-231.44,5.94,Empire,12.30,,20-May-2020 16:55
Rishair Emperor's Grace,Adachit,107.28,-163.84,44.31,Empire,8.31,,21-May-2020 02:30
Vodyanes Emperor's Grace,Aha Wa,90.38,-56.62,-17.56,Empire,9.61,,22-May-2020 02:09
Umaspi Emperor's Grace,Ahti,4.06,-79.09,-7.72,Empire,8.40,,22-May-2020 02:03
Ekono Emperor's Grace,Aiga,66.62,-147.56,102.78,Empire,18.70,,16-May-2020 08:34
HIP 101031 Emperor's Grace,Ailurii,56.38,-125.28,157.06,Empire,8.80,,14-Mar-2020 00:50
Hodack Prison Colony,Akandi,62.03,10.75,38.88,Independent,3.80,LowInf+Inf Fail,22-May-2020 02:31
Bill Turner,Alioth,-33.66,72.47,-20.66,Alliance,0.00,,22-May-2020 02:25
Turner Research Group,Alioth,-33.66,72.47,-20.66,Alliance,6.31,,2

# Experiments

In [0]:
import ipywidgets as widgets

In [12]:
combo = widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Geremy'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)
output = widgets.Output()

display(combo, output)

def on_value_change(change):
        print(change['new'])
combo.observe(on_value_change, names='value')

Combobox(value='', description='Combobox:', ensure_option=True, options=('Paul', 'John', 'George', 'Geremy'), …

Output()

In [13]:
df.describe()

,x,y,z,population,influence
count,713.000000,713.000000,713.000000,7.130000e+02,713.000000
mean,49.908398,-99.425622,-7.196748,4.708260e+08,19.553831
std,86.633145,82.848266,138.248012,2.132043e+09,18.108816
min,-352.781250,-263.656250,-952.718750,0.000000e+00,0.000000
25%,8.343750,-155.750000,-42.031250,3.109400e+04,8.200000
50%,68.531250,-116.500000,23.437500,2.563010e+05,12.600000
75%,110.000000,-50.687500,72.562500,6.403978e+06,21.221200
max,283.843750,175.000000,168.343750,2.278087e+10,100.000000


In [14]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   systemName    713 non-null    object        
 1   factionName   713 non-null    object        
 2   x             713 non-null    float64       
 3   y             713 non-null    float64       
 4   z             713 non-null    float64       
 5   allegiance    713 non-null    object        
 6   isHomeSystem  713 non-null    bool          
 7   population    713 non-null    int64         
 8   influence     713 non-null    float64       
 9   updated       713 non-null    datetime64[ns]
 10  control       713 non-null    bool          
 11  vulnerable    713 non-null    object        
dtypes: bool(2), datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 57.2+ KB


In [15]:
uncontrolled

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable
0,27 G. Caeli,Vodyanes Emperor's Grace,75.84375,-62.09375,-33.34375,Empire,False,3993953406,18.6000,2020-05-22 02:20:42,False,
1,64 Ceti,Worster Insurance,-37.06250,-100.06250,-79.12500,Independent,True,5096451,2.8000,2020-05-21 20:53:49,False,Low Influence
3,Achuar,HIP 5183 Emperor's Grace,26.09375,-231.43750,5.93750,Empire,False,2138,12.3016,2020-05-20 16:55:39,False,
4,Adachit,Rishair Emperor's Grace,107.28125,-163.84375,44.31250,Empire,False,8142435133,8.3083,2020-05-21 02:30:33,False,
5,Aha Wa,Vodyanes Emperor's Grace,90.37500,-56.62500,-17.56250,Empire,False,43924,9.6096,2020-05-22 02:09:01,False,
...,...,...,...,...,...,...,...,...,...,...,...,...
707,Col 285 Sector UE-G c11-20,Chere Emperor's Grace,124.06250,-17.78125,126.40625,Empire,False,1040097,4.5590,2020-05-21 04:41:40,False,
708,Col 285 Sector BQ-N c7-13,Sirius Luxury Transports,158.31250,69.50000,-42.03125,Independent,False,24119,13.9000,2020-05-15 14:47:47,False,Anarchy
709,Col 285 Sector AQ-P d5-52,HIP 112001 Emperor's Grace,-12.18750,-207.25000,123.37500,Empire,False,28593,8.8088,2020-05-18 18:18:50,False,
711,Delphi,Janus Incorporated,-63.59375,-147.40625,-319.09375,Independent,False,300000,8.4000,2020-05-22 01:43:07,False,


In [16]:
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [17]:
@interact
def show_factions_less_than(influence=(0, 100, 1)):
    return df.loc[df['influence'] < influence]

interactive(children=(IntSlider(value=50, description='influence'), Output()), _dom_classes=('widget-interact'…

In [18]:
@interact
def show_factions_with(IncludeHomeSystems=False, IncludeControlSystems=True, IncludeEmperorsGrace=False):
    return df.loc[(~df['isHomeSystem']|IncludeHomeSystems) & (~df['control']|IncludeControlSystems) & ( ~df[ 'factionName' ].str.contains("Emperor's Grace")|IncludeEmperorsGrace )]

interactive(children=(Checkbox(value=False, description='IncludeHomeSystems'), Checkbox(value=True, descriptio…

In [19]:
import math
df.loc[ df['x']**2 + df['y']**2 + df['z']**2< 100]

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable
376,Luhman 16,Sirius Corporation,6.3125,0.59375,1.71875,Independent,False,170509,52.9179,2020-05-22 02:13:05,True,
527,Sirius,Sirius Corporation,6.2500,-1.28125,-5.75000,Independent,True,2500000,55.8442,2020-05-22 02:14:01,True,
533,Sol,Aegis Core,0.0000,0.00000,0.00000,Independent,False,22780871769,6.6000,2020-05-22 02:26:31,False,


In [20]:
import math
a = -69.0625
b = 22.37
c = -148.09375
#a=-80.15625
#b=144.09375
#c=-333.375
df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<100')

,systemName,factionName,x,y,z,allegiance,isHomeSystem,population,influence,updated,control,vulnerable
54,BD+33 801,Sirius Hot2Cold,-31.31250,-25.03125,-107.87500,Independent,False,35800,14.2857,2020-05-21 23:34:36,False,
101,Chicoana,Sirius Hot2Cold,-35.00000,-23.21875,-110.50000,Independent,False,266162,4.9950,2020-05-19 21:47:20,False,
118,Daik,Sirius Hyperspace,-69.06250,-22.37500,-148.09375,Independent,False,2200,68.4369,2020-05-17 23:56:22,True,
172,Hemaki,Sirius Hot2Cold,-38.87500,-22.56250,-115.50000,Independent,False,2362890899,7.3926,2020-05-17 05:30:47,False,
417,Mooramba,Sirius Hyperspace,-68.62500,-17.50000,-149.12500,Independent,False,2752,46.5000,2020-05-21 17:57:36,True,
441,Ngalia,Sirius Hyperspace,-74.62500,-35.03125,-165.96875,Independent,True,9695033,5.5888,2020-05-21 17:55:51,False,
503,Ross 591,Sirius Hot2Cold,-32.18750,-28.09375,-111.37500,Independent,True,4100889,11.8118,2020-05-20 00:24:04,False,
687,HIP 17412,Sirius Hyperspace,-71.25000,-41.59375,-167.62500,Independent,False,734001,3.3000,2020-05-13 14:49:04,False,Low Influence
692,HIP 17655,Sirius Hyperspace,-83.40625,-32.90625,-178.37500,Independent,False,57282,17.9000,2020-05-20 20:23:04,False,
703,Wolf 202,Sirius Hyperspace,-70.15625,-27.59375,-140.21875,Independent,False,18776,7.2072,2020-05-20 01:09:20,False,Anarchy


In [21]:
import math
a = -69.0625
b = 22.37
c = -148.09375
dist = 150
tit = f'Club systems within {dist}ly of ({a},{b},{b})'
#a=-80.15625
#b=144.09375
#c=-333.375
fie = df.query(f'sqrt( (x-{a})**2 + (y-{b})**2 + (z-{c})**2)<{dist}')
figz = px.scatter_3d(fie, x='x', y='z', z='y',color='influence', hover_data={  'systemName', 'factionName'},title=tit)
figz.show()

In [22]:
#@title
import sys
sys.executable
sys.path

!{sys.executable} -m pip install cufflinks 
#plotly


from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [23]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cufflinks.themes.THEMES.keys()), 
                 colorscale=list(cufflinks.colors._scales_names.keys())):
    
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             #text='title',
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

interactive(children=(Dropdown(description='x', options=('x', 'y', 'z', 'population', 'influence'), value='x')…

In [29]:
import cufflinks as cf
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(1000, 2), columns=['A', 'B']).cumsum()
fig = df.iplot(asFigure=True, xTitle="The X Axis",
                    yTitle="The Y Axis", title="The Figure Title")
fig.show()